In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import openpyxl # 파이썬에서 엑셀 다루기 위한 라이브러리
from openpyxl import Workbook

import time

# options 객체를 만들지 않으면 실행시 브라우저 창이 꺼짐
options = Options()
options.add_argument("--start-maximized") #창크기 최대화
options.add_experimental_option('detach', True)

driver = webdriver.Chrome(options=options) # 크롬을 웹 브라우저로 선택

url = "https://shopping.naver.com/home" # 요 주소로 선택
driver.get(url) # 브라우저 열기

In [56]:
driver.find_element(By.CLASS_NAME, "_searchInput_search_text_83jy9").send_keys("오뚜기") # 크롬 페이지 사이즈 차이에 따라 클래스 이름이 바뀔 수 있으며 이러면 아예 안먹힐 수 있음
driver.find_element(By.CSS_SELECTOR, 
                    "#gnb-gnb > div._gnb_header_area_nfFfz > div > div._gnbContent_gnb_content_JUwjU > div._gnbSearch_gnb_search_ULxKx > form > div > div._gnbSearch_layer_content_VQK9i > div > div._searchInput_search_input_4SbSW._searchInput_open_q-L2t._searchInput_has_keyword_O9PqZ > input").send_keys("오뚜기")
#copy selector로 카피해서 send_keys하는 방법
time.sleep(2) # 그래서 f12창을 따로 뺼 필요 있음
driver.find_element(By.CLASS_NAME, "_searchInput_search_text_83jy9").send_keys(Keys.ENTER) # 검색창에 엔터키 입력해서 검색하는 방법
time.sleep(3)

In [47]:
def extract_value(data, key): # 이름이랑 값 추출 용도
    starting_point = data.find(f'"key":"{key}"') # key 값을 기준으로 그 뒤의 value 찾기 위한 준비

    start = data.find('"value":"', starting_point) + len('"value":"') # "value"의 시작점 찾기
    end = data.find('"', start)  # 값의 끝을 찾기
    return data[start:end]

In [58]:
def add_products_mini(product_list, card_class):
    products = driver.find_elements(By.CLASS_NAME, card_class) #클래스 이름에 맞춰 카드들 찾기
    for product in products: # 카드내부 정보 넣기
        data = product.get_attribute("data-shp-contents-dtl") # 데이터 목록 찾기
        prod_nm = extract_value(data, "prod_nm") # 이름 추출
        price = extract_value(data, "price") # 가격 추출
        url = products[1].get_attribute("href") # href 값은 a요소에 텍스트 하나로 되어 있음
        print("상품명:", prod_nm, "판매가:", price, "url:", url)
        product_list.append([prod_nm, price, url]) # 리스트에 집어넣기

In [62]:
def add_products(product_list, card_class):
    current_height = driver.execute_script("return window.scrollY") # 현재 창의 높이
    count = 0 # 0번째 페이지
    while True:
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END) # 페이지 끝까지 가기
        time.sleep(1)
        
        scrolled_height = driver.execute_script("return window.scrollY") # 스크롤 된 페이지 길이
        
        if (current_height == scrolled_height): # 스크롤이 끝이라면
            break
        
        if count > 1:
            break
        count+=1 # 페이지 증가
        
        current_height = scrolled_height # 현재 높이 = 스크롤 된 높이
    
    products = driver.find_elements(By.CLASS_NAME, card_class) #클래스 이름에 맞춰 카드들 찾기
    for product in products: # 카드내부 정보 넣기
        data = product.get_attribute("data-shp-contents-dtl") # 데이터 목록 찾기
        prod_nm = extract_value(data, "prod_nm") # 이름 추출
        price = extract_value(data, "price") # 가격 추출
        url = products[1].get_attribute("href") # href 값은 a요소에 텍스트 하나로 되어 있음
        print("상품명:", prod_nm, "판매가:", price, "url:", url)
        product_list.append([prod_nm, price, url]) # 리스트에 집어넣기

In [63]:
product_list = [['상품명', '가격', 'url']] # 시작 리스트
add_products_mini(product_list, "miniProductCard_link__1X65D") # 미니 카드들 넣기
add_products(product_list, "basicProductCard_link__urzND") # 일반 카드들 넣기

상품명: 오뚜기 맛있는 오뚜기밥 판매가: 23900 url: https://smartstore.naver.com/main/products/10349335572?nl-query=%EC%98%A4%EB%9A%9C%EA%B8%B0&nl-au=ba059cfc8629432e8018748b048ebcf2&NaPm=ci%3Dba059cfc8629432e8018748b048ebcf2%7Cct%3Dm4xlesoj%7Ctr%3Dnslbrd%7Csn%3D3135658%7Chk%3Dadca2eda7f4456c62ff3b078fa0316ac7b70b584
상품명: 오뚜기 맛있는 오뚜기밥 200g 18개+오뚜기밥 200g 18개 외 8종 판매가: 31900 url: https://smartstore.naver.com/main/products/10349335572?nl-query=%EC%98%A4%EB%9A%9C%EA%B8%B0&nl-au=ba059cfc8629432e8018748b048ebcf2&NaPm=ci%3Dba059cfc8629432e8018748b048ebcf2%7Cct%3Dm4xlesoj%7Ctr%3Dnslbrd%7Csn%3D3135658%7Chk%3Dadca2eda7f4456c62ff3b078fa0316ac7b70b584
상품명: 오뚜기 컵누들 매콤한맛 37.8g 15개+우동맛 38.1g 15개 외 8종 판매가: 26900 url: https://smartstore.naver.com/main/products/10349335572?nl-query=%EC%98%A4%EB%9A%9C%EA%B8%B0&nl-au=ba059cfc8629432e8018748b048ebcf2&NaPm=ci%3Dba059cfc8629432e8018748b048ebcf2%7Cct%3Dm4xlesoj%7Ctr%3Dnslbrd%7Csn%3D3135658%7Chk%3Dadca2eda7f4456c62ff3b078fa0316ac7b70b584
상품명: [4+4] 오뚜기 진라면 매운맛 120g 20개+순한맛 20개 

In [54]:
from openpyxl import Workbook

wb = Workbook()

ws = wb.active # 액티브 한걸로 선택
ws.title = 'Data' # 제목

for name,value,url in product_list: #각 데이터마다
    ws.append([name,value, url]) # 하나씩 한줄씩 붙임

wb.save('./product_list.xlsx')

In [ ]:
<a href="https://smartstore.naver.com/main/products/5820590983?nl-query=%EC%98%A4%EB%9A%9C%EA%B8%B0&amp;nl-au=ac3970d3c7a64cc280de378271b534e5&amp;NaPm=ci%3Dac3970d3c7a64cc280de378271b534e5%7Cct%3Dm4xif09y%7Ctr%3Dnslbrd%7Csn%3D3135658%7Chk%3D100fcda8489ba416c1835950cedd2f7e292cfc16" 
target="_blank" 
class="miniProductCard_link__1X65D _nlog_click _nlog_impression_element" 
aria-labelledby="brand_product_83365090397_information_5820590983" 
data-shp-page-key="100407255" data-shp-sti="shopping" data-shp-nsc="svc.shopping.plusall" data-shp-inventory="topbrd" data-shp-area="topbrd.pd" data-shp-area-type="slot" data-shp-area-id="pd" data-shp-temp="[{&quot;key&quot;:&quot;sas_logcode&quot;,&quot;value&quot;:&quot;50&quot;},{&quot;key&quot;:&quot;isps_yn&quot;,&quot;value&quot;:&quot;n&quot;}]" data-shp-contents-id="83365090397" data-shp-contents-type="nv_mid" data-shp-contents-grp="prod" data-shp-contents-rank="1" data-shp-contents-dtl="[{&quot;key&quot;:&quot;prod_nm&quot;,&quot;value&quot;:&quot;오뚜기 맛있는 오뚜기밥&quot;},{&quot;key&quot;:&quot;price&quot;,&quot;value&quot;:&quot;23900&quot;},{&quot;key&quot;:&quot;chnl_prod_no&quot;,&quot;value&quot;:&quot;5820590983&quot;},{&quot;key&quot;:&quot;cat_id&quot;,&quot;value&quot;:&quot;50012420&quot;},{&quot;key&quot;:&quot;promotion_yn&quot;,&quot;value&quot;:&quot;y&quot;},{&quot;key&quot;:&quot;nfa_type&quot;,&quot;value&quot;:&quot;일반배송&quot;}]" data-shp-contents-provider-type="mall_seq" data-shp-contents-provider-id="3135658" data-shp-contents-provider-dtl="[{&quot;key&quot;:&quot;chnl_no&quot;,&quot;value&quot;:101078690},{&quot;key&quot;:&quot;adsr_type&quot;,&quot;value&quot;:&quot;smartstore&quot;},{&quot;key&quot;:&quot;brandstore_type&quot;,&quot;value&quot;:&quot;y&quot;}]" data-shp-action-uid="ac3970d3c7a64cc280de378271b534e5"></a>

In [ ]:
<a href="https://adcr.naver.com/adcr?x=Ocplcz2wPlec7+6ou39Tyf///w==ki+yPnnJSz+MhhQ1t+XTdl0nECquUJ9VNkQdF6uGc8DGf/ZcRQWEfCULHN0IOMs4ZHb26Pffuh0hHYd9AWe1G1SLwBf3zNI5UA5uzctPYG4b8MUXMqnUtyAXn8v29EywvQ67/CX9VqgFGKmqrJHLkhf30G43LhV5X/ZIpCypflDGzEmzAn/tKj4+o9iT9MOolub0sZmDVJHarBWZhylV1JuVq1H5FExk5hPNZDARInq0jcDg6mgG3qSA7hsYLFydZvxe8LGvwJcEauscc/gQKt+O3HcI+M7zolQILoiWM2Gf6n8I3mzwyJstg+A7fXQEzPgHRZ2RWN0T/vOPasCPvcsWezg97+GXONKyhiXwLjOZe2jDWF26bAk9/SRrZmDU4LdgmVziC2tL58UNokDVRlYjBHcxXj1nIHHzYv0WVrwC8AXQNN8vaZAT1DZqzDrTrEAnP/VokTpPQSBzuRdJmGoEdOpNqJaKkc/dDyZhvX9MRIgnY8EDK9//Tv2sYZSruTwWDMUPwbiiCa5OZ/sm3Tjx2mbXX6EkzXqVgcGiGB4dXXwgZeNJStlp3MG3tKKBVCdRlb2leEF7grsB80o89zDKM30ccZT8LTRTYX0hJlzTCQzld711G0VgafIHDYFIBX0avb+49qruITMle8YIUx6/3LNRYNYixbqm49w7wqUFg7v+46GhzqdR9AeV3udzxjwJq9ctc0FWgGsk7pEHFBSTeQziv4P2MzU2ciN4rD9Wqp6+77Lcc6bvdFE53bxYe74IepsgvuShzqiHbFXNneIkQerecBjuA7iHJ/bEYSJuCwwSCf5EGSkRep+ppY50AA25mMF8KWbmRlq9wZWGCOATHy6Qp6/8PLufn2IjMUDClM6Oo8zvbxKw+yne9i5yMBF3eTpY1WZlqSG89gMVkTHHmUqXbYtQRXNjm6v5t2VeCHYM8hUZJ0vQsylLhOmwz+kyG79VrJX8EGp0jVUleTfT53gquT2LWXZ10njNZHiGH5MfiuYZpbdm1OnTpcI6J6tDAM/IBEbkj9Nxqxd9lwTR7yiv+pirEafPnC2K6O+I=#channelProductNo=6059761929" 
target="_blank" class="basicProductCard_link__urzND _nlog_click _nlog_impression_element" 
aria-labelledby="basic_product_card_information_6059761929" data-shp-page-key="100407255" data-shp-sti="shopping" data-shp-nsc="svc.shopping.plusall" 
data-shp-abt_exps="[{&quot;serId&quot;:&quot;shopping&quot;,&quot;type&quot;:&quot;oep&quot;,&quot;expId&quot;:&quot;PWL-AD-BLIND&quot;,&quot;varId&quot;:&quot;10&quot;,&quot;value&quot;:{&quot;bt&quot;:&quot;1&quot;,&quot;bucket&quot;:&quot;1&quot;,&quot;for&quot;:&quot;impression-neo&quot;,&quot;isExtraBucket&quot;:&quot;true&quot;,&quot;is_control&quot;:true},&quot;userType&quot;:&quot;nnb&quot;,&quot;provId&quot;:&quot;&quot;,&quot;sesnId&quot;:&quot;&quot;},{&quot;serId&quot;:&quot;shopping&quot;,&quot;type&quot;:&quot;oep&quot;,&quot;expId&quot;:&quot;SA-RSCORE&quot;,&quot;varId&quot;:&quot;4&quot;,&quot;value&quot;:{&quot;bucket&quot;:&quot;0&quot;,&quot;for&quot;:&quot;impression-neo&quot;,&quot;site&quot;:&quot;naver.search.pc.pl^mnaver.tab.m.all&quot;,&quot;is_control&quot;:true},&quot;userType&quot;:&quot;nnb&quot;,&quot;provId&quot;:&quot;&quot;,&quot;sesnId&quot;:&quot;&quot;}]" 
data-shp-inventory="prod" data-shp-order_con="[{&quot;key&quot;:&quot;sort&quot;,&quot;value&quot;:&quot;추천순&quot;},{&quot;key&quot;:&quot;resort&quot;,&quot;value&quot;:&quot;ON&quot;}]" data-shp-area="prod.pd" data-shp-area-type="slot" data-shp-area-id="pd" data-shp-area-dtl="[{&quot;key&quot;:&quot;view_type&quot;,&quot;value&quot;:&quot;2&quot;}]" data-shp-temp="[{&quot;key&quot;:&quot;isps_yn&quot;,&quot;value&quot;:&quot;n&quot;}]" 
data-shp-contents-id="nad-a001-02-000000278065350" data-shp-contents-type="SA_prod" data-shp-contents-grp="ad" data-shp-contents-rank="1" data-shp-contents-dtl="[{&quot;key&quot;:&quot;chnl_prod_no&quot;,&quot;value&quot;:&quot;6059761929&quot;},{&quot;key&quot;:&quot;nv_mid&quot;,&quot;value&quot;:&quot;83604261417&quot;},{&quot;key&quot;:&quot;click_url&quot;,&quot;value&quot;:&quot;https://smartstore.naver.com/main/products/6059761929&quot;},{&quot;key&quot;:&quot;price&quot;,&quot;value&quot;:&quot;33500&quot;},{&quot;key&quot;:&quot;cat_id&quot;,&quot;value&quot;:&quot;50014021&quot;},{&quot;key&quot;:&quot;promotion_yn&quot;,&quot;value&quot;:&quot;n&quot;},{&quot;key&quot;:&quot;nfa_type&quot;,&quot;value&quot;:&quot;일반배송&quot;},{&quot;key&quot;:&quot;prod_nm&quot;,&quot;value&quot;:&quot;오뚜기 옛날국수 중면 3kg 4개/box&quot;}]" 
data-shp-contents-provider-type="mall_seq" data-shp-contents-provider-id="1023687" data-shp-contents-provider-dtl="[{&quot;key&quot;:&quot;chnl_no&quot;,&quot;value&quot;:100506110},{&quot;key&quot;:&quot;adsr_type&quot;,&quot;value&quot;:&quot;smartstore&quot;}]" data-shp-action-uid="827d5f3a7778445fa83dc0c1e7c11180"></a>

In [ ]:
# 다른 방식의 해결법
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

import time

driver = webdriver.Chrome(options=options)  
driver.get("https://shopping.naver.com/home")
time.sleep(2)

search = driver.find_element(By.CSS_SELECTOR, '#gnb-gnb > div._gnb_header_area_nfFfz > div > div._gnbContent_gnb_content_JUwjU > div._gnbSearch_gnb_search_ULxKx > form > div > div > div > div > input')
search.click()
search.send_keys('사과')

search_btn = driver.find_element(By.CSS_SELECTOR, '#gnb-gnb > div._gnb_header_area_nfFfz > div > div._gnbContent_gnb_content_JUwjU > div._gnbSearch_gnb_search_ULxKx > form > div > div > div > div > button._searchInput_button_search_wu9xq._nlog_click > svg')
search_btn.click()
time.sleep(2)

current_height = driver.execute_script("return window.scrollY")

count = 0

while True:
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
    time.sleep(1)
    
    scrolled_height = driver.execute_script("return window.scrollY")
    
    if (current_height == scrolled_height):
        break
    
    if count > 1:
        break
    count+=1
    
    current_height = scrolled_height

import openpyxl
# 엑셀 만들기
wb = openpyxl.Workbook()

# 워크시트 만들기
ws = wb.active
ws.title = '쇼핑'

# 데이터 추가하기
ws.append(['상품명', '판매가', 'url'])

# 엑셀 저장하기
filePath = 'shopping.xlsx'

products = driver.find_elements(By.CSS_SELECTOR, '.basicProductCard_basic_product_card__TdrHT')

for product in products:
    # print("product",product)
    t = product.find_element(By.CSS_SELECTOR, '.basicProductCardInformation_title__Bc_Ng').text
    print(t)
    p = product.find_element(By.CSS_SELECTOR, '.priceTag_inner_price__TctbK').text.split()[0]    
    print(p)
    a = product.find_element(By.CSS_SELECTOR, '.basicProductCard_link__urzND')
    # print(a)
    l = a.get_attribute("href")
    print(l)
    ws.append([t, p, l])

wb.save(filePath)